In [1]:
!pip install -q langchain faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.5 MB/s eta 0:00:00


In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import json

# Load JSON data
with open("/content/corvit_faq_reformatted (1).json", "r") as f:
    raw_docs = json.load(f)

# Convert to LangChain Documents
docs = [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in raw_docs]

# Use a good embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Build FAISS index
db = FAISS.from_documents(docs, embedding_model)

# Save retriever for reuse
db.save_local("corvit_faiss_index11")


In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import json

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipython-input-3-675302313.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
faiss_path = "/content/drive/MyDrive/CorvitChatbot37/corvit_faiss_index11"
retriever = FAISS.load_local(
    folder_path=faiss_path,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
).as_retriever(search_type="similarity", k=3)

# Urdu or English query
query = 'duration of AI course?'
# Optionally translate (we'll keep it as-is for now)
results = retriever.get_relevant_documents(query)

# Display top results
for i, doc in enumerate(results):
    print(f"Result {i+1}:")
    print("🔹 Question:", doc.page_content)
    print("✅ Answer:", doc.metadata.get("answer", "No answer found"))
    print("-" * 60)

Result 1:
🔹 Question: What is the duration of the  course?
✅ Answer: The course typically lasts 2 to 3 months, depending on the batch schedule and learning pace.
------------------------------------------------------------
Result 2:
🔹 Question: Can you explain what the duration of the  course is?
✅ Answer: The course typically lasts 2 to 3 months, depending on the batch schedule and learning pace.
------------------------------------------------------------
Result 3:
🔹 Question: Outline the Artificial Intelligence Associate course curriculum at Corvit?
✅ Answer: Corvit’s Artificial Intelligence course covers machine learning fundamentals, deep learning, neural networks, natural language processing, TensorFlow/PyTorch, data preprocessing, model evaluation, and real‑world AI applications.
------------------------------------------------------------
Result 4:
🔹 Question: I'd like to know what the duration of the  course means.
✅ Answer: The course typically lasts 2 to 3 months, depending 

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/drive/MyDrive/raft_model_chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [6]:
import re

def clean_output(text):
    # Step 1: Try to extract text after "A:" if present
    match = re.search(r"A:\s*(.*)", text, re.DOTALL)
    if match:
        text = match.group(1).strip()

    # Step 2: Remove leading "Q: ..." lines (if any)
    text = re.sub(r"Q:.*?\n", "", text).strip()

    # Step 3: Remove repeated phrases, "Therefore..." trails, and excess punctuation
    text = re.sub(r'(##end_quote##|\bTherefore\b.*?)$', '', text.strip(), flags=re.IGNORECASE)

    # Step 4: Normalize spacing
    text = re.sub(r'\s+', ' ', text)

    return text.strip()


In [13]:
from sentence_transformers import SentenceTransformer, util
import torch
# === Reranker model ===
reranker_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# === Domain restriction helpers ===
def is_out_of_domain(query):
    blacklist = ["google", "amazon", "facebook", "pakistan politics", "elon musk"]
    return any(word in query.lower() for word in blacklist)

def is_outside_islamabad(query):
    cities = ["lahore", "karachi", "multan", "peshawar", "faisalabad"]
    return any(city in query.lower() for city in cities)

# === Main QA function ===
def generate_answer(query):
    if is_out_of_domain(query):
        return "🤖 Sorry, I’m only trained to answer questions about Corvit Islamabad."

    if is_outside_islamabad(query):
        return "🤖 I’m focused only on Corvit Islamabad. I don’t have data for other branches."

    # Step 1: Retrieve documents
    results = retriever.get_relevant_documents(query)

    if not results:
        return "🤖 Sorry, I couldn’t find any relevant answer."

    # Step 2: Rerank using sentence similarity
    query_emb = reranker_model.encode(query, convert_to_tensor=True)
    doc_texts = [doc.page_content for doc in results]
    doc_embs = reranker_model.encode(doc_texts, convert_to_tensor=True)
    sims = util.cos_sim(query_emb, doc_embs)[0]

    ranked = sorted(zip(sims, results), key=lambda x: x[0], reverse=True)

    top_sim, top_doc = ranked[0]
    top_answer = top_doc.metadata.get("answer", "").strip()

    # If high confidence or short answer → return directly
    if top_sim.item() > 0.65 or len(top_answer.split()) < 12:
        return clean_output(top_answer)

    # Else: use generative model with top-2 results as context
    context = ""
    for sim, doc in ranked[:2]:
        context += f"Q: {doc.page_content}\nA: {doc.metadata.get('answer', '')}\n\n"

    prompt = f"""You are a helpful assistant answering user questions based on the context provided.

Context:
{context}
Now, answer this:

User Question: {query}
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200, num_beams=4)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return clean_output(response)

# === Run chatbot loop ===
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        answer = generate_answer(user_input)
        print("Bot:", answer)


You: what is corvit?
Bot: Corvit system is a leading IT training and consulting company offering professional certifications and skill-based courses.For over 15 years, the name “Corvit” has stood as a symbol of excellence and innovation in IT education, particularly in the field of Computer Networking. Founded with a mission to elevate technical standards across the region, Corvit system has grown into one of the most respected IT training organizations in Pakistan and the South Asian subcontinent.
You: how to register in web devlopment course?
Bot: You can enroll in Corvit courses by following these steps: 1. Visit the official website: https://www.corvit.com 2. Choose your nearest branch (e.g., Islamabad, Lahore, Karachi). 3. Browse available courses such as Cyber Security, Networking (CCNA, CCNP, CCIE), AI, Web & App Development, and more. 4. Contact the branch directly. For example, Islamabad branch: - Phone: 051-111-333-999 - Email: info.isb@corvit.com 5. Speak with an admission a